In [ ]:
#@title # 初始化（要先執行）
#@markdown 先下載必要套件
!rm -rf /content/sample_data
!apt install -y -qq aria2
!pip install -qq gigafile mega.py
!mkdir -p /content/DL
mega_count = 0
%cd /content/DL

In [ ]:
#@title # 👇 **下載**
import re
%cd /content/DL
link = "" #@param{type:"string"}
from_direct_link = False #@param{type:"boolean"}
from_torrent = False #@param{type:"boolean"}
from_gigafile = False #@param{type:"boolean"}
from_pixeldrain = False #@param{type:"boolean"}
from_mega = False #@param{type:"boolean"}

if from_torrent:
  !aria2c --summary-interval=10 --seed-time=0 --allow-overwrite=true "{link}"

if from_direct_link:
  !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{link}"

if from_gigafile:
  !gfile download "{link}"

if from_pixeldrain:
  url_check = re.match('https://pixeldrain.com/u/', link)
  if url_check:
    pd_id = link[url_check.span()[1]: len(link)]
    bypass = 'https://pd.cybar.xyz/' + pd_id
    !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{bypass}"
  else:
    print("URL格式錯誤！")
    print("正確格式應為`https://pixeldrain.com/u/XXXXXXXX`")

if from_mega:
  if mega_count == 0:
    mega_count = 1
    from mega import Mega
    mega = Mega()
    m = mega.login()
  m.download_url(link)

print("\nDone!")

In [ ]:
#@title # 👇 **上傳**
from google.colab import drive
import os, json, re

google_drive = False #@param{type:"boolean"}
google_drive_path = "ColabDownloads/" #@param {type:"string"}
#@markdown **資料夾位置記得加 "/"**

#@markdown ---
#@markdown 以下皆需填filename(path也行)

#@markdown 有錯誤時可能是檔名中有特殊符號
filename = "" #@param{type:"string"}
#@markdown ---
#@markdown **Pixeldrain上限 [20](https://pixeldrain.com/) GB**

#@markdown 現在需要帳號ㄌ！
Pixeldrain = False #@param{type:"boolean"}
pd_api_key = "" #@param{type:"string"}
#@markdown ---
Gigafile = False #@param{type:"boolean"}
#@markdown ---
#@markdown **catbox上限 [200](https://catbox.moe/) MB**
catbox = False #@param{type:"boolean"}
#@markdown ---
Buzzheavier = False #@param{type:"boolean"}
#@markdown ---
Gofile = False #@param{type:"boolean"}


if google_drive:
  if not os.path.exists('/content/gdrive'):
    drive.mount('/content/gdrive')
  !rsync -avh --progress . "/content/gdrive/MyDrive/"{output_path}
  print("\nDone!")

if Pixeldrain:
  if pd_api_key == "":
    print("請填入API密鑰！")
    print("https://pixeldrain.com/user/api_keys")
  else:
    pdupload = !curl -T "{filename}" -u :$pd_api https://pixeldrain.com/api/file/
    res = json.loads(pdupload[0])
    # output.clear()
    print("下載網址： https://pixeldrain.com/u/", res.get('id'), sep='')

if Gigafile:
  !gfile -p -n 24 upload "{filename}"

if catbox:
  !curl -F "reqtype=fileupload" -F "fileToUpload=@{filename}" https://catbox.moe/user/api.php

if Buzzheavier:
  BuzzUpUrl = "https://w.buzzheavier.com/t/" + filename
  BuzzUp = !curl -#o - -T "$filename" "$BuzzUpUrl" | cat
  print("下載網址： https://buzzheavier.com/f/", json.loads(BuzzUp[-1]).get('id'), sep='')

if Gofile:
  server = !curl -X GET 'https://api.gofile.io/servers'
  good_server = json.loads(server[0]).get('data').get('servers')[0].get('name')
  go_url = f"https://{good_server}.gofile.io/contents/uploadfile"
  go_up = !curl -X POST $go_url -F "file=@{filename}"
  print('下載網址：', json.loads(go_up[0]).get('data').get('downloadPage'))

In [ ]:
#@title # 👇 **移除下載檔案 (清理一下colab)**
!rm -rf /content/DL/*
%cd /content/DL
print("乾淨ㄌ")

---
# 以下小工具


In [ ]:
#@title # Colab終端機
#@markdown [colab-xterm](https://github.com/InfuseAI/colab-xterm)
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
#@title # Telegram 工具
#@markdown [tdl](https://github.com/iyear/tdl) [文件](https://docs.iyear.me/tdl/)
!curl -sSL https://docs.iyear.me/tdl/install.sh | sudo bash
# upload tdl
# unzip tdl.zip -d ~/.tdl

In [ ]:
#@title # Mega帳號產生器
#@markdown [py_mega_account_generator](https://github.com/qtchaos/py_mega_account_generator)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get update
!apt-get install ./google-chrome-stable_current_amd64.deb
!rm google-chrome-stable_current_amd64.deb
!git clone https://github.com/qtchaos/py_mega_account_generator.git
%cd py_mega_account_generator
!pip install -r requirements.txt
%cd ..
# 第一次執行把這個複製並貼上`/opt/google/chrome/chrome`

In [ ]:
#@title # YouTube影片下載
#@markdown 下載格式會是`影片id.mkv`，這樣比較不會有問題
from google.colab import output
import json
!pip install yt-dlp
yt_url = "https://youtu.be/dQw4w9WgXcQ" #@param {type:"string"}
!yt-dlp --ignore-errors --output "%(id)s.%(ext)s" --verbose --merge-output-format mkv --embed-thumbnail --embed-metadata --embed-info-json --concurrent-fragments 24 $yt_url
server = !curl -X GET 'https://api.gofile.io/servers'
good_server = json.loads(server[0]).get('data').get('servers')[0].get('name')
go_url = f"https://{good_server}.gofile.io/contents/uploadfile"
VT = !yt-dlp --simulate --print "%(id)s" $yt_url
FN = VT[0] + '.mkv'
go_up = !curl -X POST $go_url -F "file=@{FN}"
output.clear()
print('下載網址：', json.loads(go_up[0]).get('data').get('downloadPage'))

In [ ]:
#@title # 巴哈姆特自動簽到+抽獎
#@markdown 從我另一個 [project](https://colab.research.google.com/github/chikenscrach/myba/blob/main/BA.ipynb) 搬過來的ㄏ
from google.colab import output, files
import threading
!apt install nodejs
!npm install -g n
!n v19.9.0


def job():
  !npm install -g bahamut-automation


t = threading.Thread(target=job)
t.start()
!npm install playwright
!npx playwright install
t.join()
!rm /usr/local/lib/node_modules/bahamut-automation/dist/modules/ad_handler/index.js
!rm /usr/local/lib/node_modules/bahamut-automation/dist/modules/del_mail/index.js
!rm /usr/local/lib/node_modules/bahamut-automation/dist/modules/login/index.js
!rm /usr/local/lib/node_modules/bahamut-automation/dist/modules/lottery/index.js
!wget https://raw.githubusercontent.com/chikenscrach/myba/main/ad_handler/index.js -P /usr/local/lib/node_modules/bahamut-automation/dist/modules/ad_handler
!wget https://raw.githubusercontent.com/chikenscrach/myba/main/del_mail/index.js -P /usr/local/lib/node_modules/bahamut-automation/dist/modules/del_mail
!wget https://raw.githubusercontent.com/chikenscrach/myba/main/login/index.js -P /usr/local/lib/node_modules/bahamut-automation/dist/modules/login
!wget https://raw.githubusercontent.com/chikenscrach/myba/main/lottery/index.js -P /usr/local/lib/node_modules/bahamut-automation/dist/modules/lottery
output.clear()
uploaded = files.upload()
filename = next(iter(uploaded))
!ba -c "{filename}"

In [ ]:
%cd /content
#@title # Firefox瀏覽器
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Firefox_logo%2C_2019.svg/330px-Firefox_logo%2C_2019.svg.png' height="100" alt="Firefox"/></center>
#@markdown <center><h3><a href="https://github.com/jlesage/docker-firefox">jlesage/firefox</a>, <a href="https://github.com/jlesage/docker-firefox-esr">firefox-esr</a></h3></center>
#@markdown <center>抄來ㄉ <a href="https://colab.research.google.com/github/biplobsd/OneClickRun/blob/master/OneClickRun.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></center><br>

import os, tarfile
import urllib.request
from IPython.display import clear_output
import subprocess
import socket
import time

#####################################
USE_FREE_TOKEN = True # 免費token

TOKEN = ""  # 有需要再用，很常出問題
REGION = "JP" # 選擇地區 ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" # 連線方式 ["ngrok", "localhost", "argotunnel"]
TYPE = "firefox" # 版本 ["firefox", "firefox-esr"]
CONFIG_DIRECTORY = "/content/tools/firefox"  # 設定檔位置
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
    read_subprocess_output,
    textAn
)

def is_port_open(port):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(2)
            s.connect(("localhost", port))
        return True
    except (socket.timeout, ConnectionRefusedError):
        return False

def in_output(command, output):
    try:
        result = subprocess.run(command.split(), capture_output=True, text=True, check=True)
        return output in result.stdout
    except subprocess.CalledProcessError:
        return False

def check_log_until_line_appears(command, line_to_check):
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    while True:
        line = process.stdout.readline().decode('utf-8')
        if line_to_check in line:
            return True
        elif line == '':
            return False
        else:
            time.sleep(0.1)

def popen(cmd, description=None):
    try:
        if description:
          clear_output()
          loadingAn()
          textAn(description, 'twg')
        subprocess.Popen(cmd.split()).wait()
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {cmd}")
        print(f"Error message: {e}")
        raise

loadingAn()
os.makedirs(CONFIG_DIRECTORY, exist_ok=True)

if not os.path.exists("/usr/local/bin/udocker"):
    popen('pip install udocker', "正在安裝 udocker ...")
    popen('udocker --allow-root install', '設定 udocker ...')

if not in_output('udocker --allow-root images', f'jlesage/{TYPE}:latest'):
    popen(f'udocker --allow-root pull jlesage/{TYPE}', f"獲取 jlesage/{TYPE} image ...")

if not in_output('udocker --allow-root ps', TYPE):
    popen(f'udocker --allow-root create --name={TYPE} jlesage/{TYPE}', f"創建 {TYPE} container ...")

command = f'udocker --allow-root run -v {CONFIG_DIRECTORY}:/config -p 5800:5800 {TYPE}'.split()
line_to_check = '[supervisor  ] all services started.'

done = True
if (not findProcess(TYPE, '/config/profile')) or (not is_port_open(5800)):
    popen('fuser -k -n tcp 5900')
    clear_output()
    loadingAn()
    textAn("等待所有系統開始運行 ...", 'twg')
    done = check_log_until_line_appears(command, line_to_check)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
if done:
    server = PortForward_wrapper(
        PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['firefox', '5800', 'http']],
        REGION.lower(),
        [f"{HOME}/.ngrok2/firefox.yml", 58009]
    ).start('firefox', displayB=True)
else:
    print("Error: 請終止執行階段，並重新執行。")
